Hello，同学们，你掌握本节课的知识了吗？现在来实现一个简易的中文输入法吧！巩固一下知识。

本节课的作业要求：

+ 熟练掌握n-gram语言模型

+ 认真阅读注释，对你做对该题至关重要

+ 作业已经给出大部分程序实现代码，你只需要在`######## your code ~n line ########` 与 `######## your code end ########` 行之间根据提示补充完毕相应的代码即可。


数据集：
+ 我们使用一个中文新闻分类的数据集，大家可以通过这里来下载。[点击我下载](https://pan.baidu.com/s/1oTdl_Swo5z0mDdGxNa4bwg) 提取码:5hm5
+ 数据集一共由三个文件组成，分别是`train.txt`、`dev.txt`、`test.txt`，共20万条，文件中每一行中由文本和其对应的类别id组成，两者之间使用`\t`分开，这里只有前面的文本内容对本任务有用。
+ 下载后解压缩，放在本作业notebook相同目录下即可。

In [1]:
import os
import numpy as np

## 1.读取语料

In [2]:
# 加载语料
def load_corpus():
    texts = []
    base = './THUCNews/'
    # os.listdir(base) 列出base目录下的所有文件名
    files = os.listdir(base)
    for name in files:
        if not name.endswith('.txt'):
            continue
        with open(base + name, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            # 使用.split('\t')分开标题与类别id
            texts += [lin.strip().split('\t')[0] for lin in lines]
    return texts

In [3]:
texts = load_corpus()
'一共%s条文本' % len(texts)

'一共200000条文本'

## 2.为每条句子前后添加特殊标记

为了让我们的N-gram语言模型能够对所有有限长的句子都赋予一个概率，我们面临一个小问题：怎么建模句子的结束？一种办法是我们有一个模型来建模句子的长度n，但是更加简单的办法是引入一个特殊的句子结束标签`</s>`来表示句子的结束。从生成模型的角度来说，我们先预测第一个词，然后用第一个词预测第二个词(假设是bigram)，然后用第二个词预测第三个词，……，直到遇到`</s>`，我们结束这个过程。这样的模型能够保证所有可能句子的概率加起来是1，也就是一个合法的概率分布。

但是前面一个问题就是预测第一个词没有history，虽然我们记为$p(w_{1})$，但是它不是词$w_{1}$出现的概率，而是词$w_{1}$出现在第一个位置的概率！为了统一记号，我们引入一个特殊的句子开始标签`<s>`，这样记为$p(w_{1} \vert <s>)$，它表示词$w_{1}$出现在`<s>`后的概率，也就是$w_{1}$出现在第一个位置的概率。

### 问题一：为每条语料前后添加特殊标记开头添加`<s>`结尾添加`</s>`。

In [4]:
def pad_both_ends(texts):
    pad_texts = []
    for text in texts:
        text = list(text) # 拆分字符，这里按字力度实现语言模型，例子：中国留学生->['中','国','留','学','生']
        # 提示：list().insert(index, 'something') 是为列表中index位置添加一个元素；
        # list().append()是在列表最后追加一个元素
        ######## your code ~2 line ######## 
        text.insert(0, '<s>')  
        text.append('</s>')
        ######## your code end ########
        pad_texts.append(text)
    return pad_texts

In [5]:
pad_texts = pad_both_ends(texts)
pad_texts[1]

['<s>',
 '6',
 '0',
 '年',
 '铁',
 '树',
 '开',
 '花',
 '形',
 '状',
 '似',
 '玉',
 '米',
 '芯',
 '(',
 '组',
 '图',
 ')',
 '</s>']

## 3.统计语料中字的频率，生成字与id、id与字的映射关系

### 问题二：统计语料中字的频率；字与id的映射。

In [6]:
def build_vocab(pad_texts):
    # word2freq：key是字，value是该字对应的出现频率。
    word2freq = {}
    for pad_text in pad_texts:
        for w in pad_text:
            # 提示：判断当前字（w）是否在word2freq的keys中存在，不存在就添加w，对应的value为1
            # 存在，把key（w）对应的value加1
            ######## your code ~4 line ######## 
            if w not in word2freq.keys():
                word2freq[w] = 1
            else:
                word2freq[w] += 1
            ######## your code end ######## 
    # word2id：key是字，value是该字对应的id。
    word2id = {}
    for key, _ in word2freq.items():
        # 提示：word2id中value的id从0开始增加
        # 每次设置key的值为word2id的长度
        ######## your code ~1 line ######## 
        word2id[key] = len(word2id)
        ######## your code end ######## 
    # id2word: list中每个元素是字，其对应的索引是字的id
    id2word = []
    for word, _ in word2id.items():
        id2word.append(word)
    return word2freq, word2id, id2word

In [7]:
vocab2freq, word2id, id2word = build_vocab(pad_texts)

## 4.创建转移矩阵

### 问题三：定位共现词的坐标，把对应坐标上的数值+1

In [8]:
def transfer_matrix(pad_texts, word2id):
    # 创建一个全零的矩阵，矩阵位置(i,j)对应是，word2id中id为i和id为j的两个字在语料中同时出现了多少次，区分前后关系，i字在前j字在后
    matrix = np.zeros((len(word2id), len(word2id)))
    for pad_text in pad_texts:
        # 计算句子长度
        seq_len = len(pad_text)
        for i in range(seq_len - 1):
            # 2-gram中的前后字
            pre_word = pad_text[i]
            next_word = pad_text[i + 1]
            # 前后字在字典中的id
            # word2id 是字与id的对应关系
            ######## your code ~2 line ######## 
            pre_id = word2id[pre_word]
            next_id = word2id[next_word]
            ######## your code end ######## 
            # 设置matrix对应的位置数值+1
            ######## your code ~1 line ######## 
            matrix[pre_id][next_id] += 1
            ######## your code end ######## 
    return matrix

In [9]:
matrix = transfer_matrix(pad_texts, word2id)
matrix.shape

(4803, 4803)

## 5.预测下一个字

提示：该部分你没有看到任何关于条件概率的计算代码。我们是根据给出的某个字（$w_{i}$），去预测下一个最大可能出现的字（$w_{i+1}$），根据语言模型计算公式：

$P(w_{i+1}|w_{i})=$ $count(w_{i}, w_{i+1}) \over count(w_{i})$，这里$w_{i}$是已知的，$w_{i+1}$是未知的，$w_{i+1}$的取值可能是词表中任意一个，我们只需要找出那个与$w_{i}$共同出现的次数最多的$w_{i+1}$即可。（在实现的过程中我们设置了个topk的参数，即找出最有可能出现的前k个）

### 问题四：根据matrix矩阵，统计没有在某个字后面出现过的字的个数。

In [ ]:
def next_input(word, matrix, word2id, id2word):
    # topk 给定前一个字，预测下面最有可能出现的topk个字
    topk = 10
    # 获取word对应的id
    try:
        word_id = word2id[word]
    except Exception as e:
        print(e)
        return 'oov'
    # 取出word对应matrix中的那一行数据
    row = matrix[word_id]
    # 计算字典中有多少字从来没有出现在word之后
    # 提示：row中为0的元素对应的索引，都是没有出现在word之后的字的id，可以把等于0的数量进行累加
    ######## your code ~1 line ######## 
    un_appare = (row == 0.0).sum()
    ######## your code end ######## 
    # 函数argsort()把一个array向量按照元素从小到大排列，并返回排序前对应的索引id。
    # 例子np.array([2, 5, 2, 1, 0]).argsort()，返回的是[4 3 0 2 1]，最小的元素是0，对应的位置索引id为4
    # [::-1]把一个向量反转。例子：[1,2,3][::,-1]->[3,2,1]
    # 出现在一个字之后的字是有限个的，有可能出现小于topk的情况，所以这里使用[:-un_appare]截取掉没有出现在word之后字的id
    topk_word_id = matrix[word_id].argsort()[::-1][:-un_appare][:topk]
    return {i: id2word[id] for i, id in enumerate(topk_word_id)}

## 6.测试

In [ ]:
word = input("请输入:")
while True:
    res = next_input(word[-1], matrix, word2id, id2word)
    if res == 'oov':
        word = word[:-1]
        word += input("你输入的字不存在，请重新输入:")
        continue
    print(res)
    id = input("请选择下一个词，输入对应的id，如果提供的列表中不存在接下来你想要的输入，请输入-1。")
    if int(id) != -1:
        next_word = res[int(id)]
        if next_word == "</s>":
            break
    else:
        next_word = input("请输入你想要的字：")
        if next_word == "</s>":
            break
    word += next_word
    print(word)
    print('-'*60)
print("你输入的搜索内容是:{}".format(word))